In [1]:
import  pandas as pd
from  pandasql import sqldf
import plotly.express as px
import datetime

pd.options.display.max_rows = 50
pd.options.display.max_columns = 100


pysqldf = lambda q: sqldf(q, globals())

ImportError: Unable to import required dependencies:
numpy: Error importing numpy: you should not try to import numpy from
        its source directory; please exit the numpy source tree, and relaunch
        your python interpreter from there.

In [2]:
def load_format_df():
    ''' Format DF for further processing
    '''
    df = pd.read_csv('spy_5min.csv')
    df = df.iloc[:,1:] # trim first extra column (redundant)
    df['time_converted'] = pd.to_datetime(df['time_converted'])
    # set only day session filter
    day_session = True
    if day_session:
        begin_time = datetime.time(9,45,0)
        end_time = datetime.time(15,55,0)
        df = df[(df['time_converted'].dt.time <= end_time) & (df['time_converted'].dt.time >= begin_time )]
    return df
df = load_format_df()


In [31]:
def cleanse_df(df):
    '''
    Apply some custom logic for further processing
    '''
    df =  pysqldf('''
                    WITH t as (
                        select *, DATE(time_converted) as date from df s where TIME(time_converted) BETWEEN '09:45:00' and '15:50:00'
                        ),
                        t1 as (
                        SELECT DISTINCT date,date_open,date_close from (
                            select date
                            ,FIRST_VALUE(open) OVER (PARTITION BY date ORDER BY DATETIME(time_converted) ASC) as date_open
                            ,LAST_VALUE(close) OVER (PARTITION BY date ORDER BY DATETIME(time_converted) ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING ) as date_close
                        from t))
                        select t.*, t1.date_open, t1.date_close from t left join t1 on t.date=t1.date
                   ''')
    df['time_converted'] = pd.to_datetime(df['time_converted'])
    df['high_pct_open_abs'] = abs((df['high']/df['date_open'])-1)*100
    df['low_pct_open_abs'] = abs(df['low']/df['date_open'] -1)*100
    df['weekday'] = df['time_converted'].dt.day_name()
    temp_df = df.groupby('date')[['high_pct_open_abs','low_pct_open_abs']].max()
    temp_df['abs_max_daily'] = temp_df[['high_pct_open_abs','low_pct_open_abs']].max(axis=1)
    df = df.merge(temp_df, on = 'date', suffixes=('', '_max_daily'))
    return df
df1 = cleanse_df(df)
df1

,volume,volume_weighted,open,close,high,low,timestamp,number of trades,time_converted,date,date_open,date_close,high_pct_open_abs,low_pct_open_abs,weekday,high_pct_open_abs_max_daily,low_pct_open_abs_max_daily,abs_max_daily
0,1997429.0,383.8575,385.29,383.5000,385.40,383.0200,1672757100000,14407,2023-01-03 09:45:00,2023-01-03,385.29,381.099,0.028550,0.589167,Tuesday,1.806432,1.935944,1.935944
1,1800303.0,383.0406,383.49,382.8001,383.96,382.5000,1672757400000,10841,2023-01-03 09:50:00,2023-01-03,385.29,381.099,0.345195,0.724130,Tuesday,1.806432,1.935944,1.935944
2,982402.0,382.2145,382.80,381.7100,383.10,381.6601,1672757700000,10002,2023-01-03 09:55:00,2023-01-03,385.29,381.099,0.568403,0.942122,Tuesday,1.806432,1.935944,1.935944
3,1600548.0,382.3233,381.77,382.1200,382.58,381.6800,1672758000000,11112,2023-01-03 10:00:00,2023-01-03,385.29,381.099,0.703366,0.936957,Tuesday,1.806432,1.935944,1.935944
4,879537.0,382.2191,382.12,381.9700,382.41,381.7801,1672758300000,7210,2023-01-03 10:05:00,2023-01-03,385.29,381.099,0.747489,0.910976,Tuesday,1.806432,1.935944,1.935944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32527,581365.0,566.5898,566.55,566.8401,566.92,566.4500,1727983800000,4730,2024-10-03 15:30:00,2024-10-03,567.27,567.340,0.061699,0.144552,Thursday,0.446436,0.340226,0.446436
32528,411034.0,566.8564,566.86,566.9200,566.98,566.7600,1727984100000,4637,2024-10-03 15:35:00,2024-10-03,567.27,567.340,0.051122,0.089904,Thursday,0.446436,0.340226,0.446436
32529,698426.0,567.0924,566.93,566.5600,566.95,566.4400,1727984400000,6222,2024-10-03 15:40:00,2024-10-03,567.27,567.340,0.056411,0.146315,Thursday,0.446436,0.340226,0.446436
32530,1207322.0,567.0179,566.54,567.1500,567.22,566.5400,1727984700000,8118,2024-10-03 15:45:00,2024-10-03,567.27,567.340,0.008814,0.128687,Thursday,0.446436,0.340226,0.446436


In [32]:
df1.columns

Index(['volume', 'volume_weighted', 'open', 'close', 'high', 'low',
       'timestamp', 'number of trades', 'time_converted', 'date', 'date_open',
       'date_close', 'high_pct_open_abs', 'low_pct_open_abs', 'weekday',
       'high_pct_open_abs_max_daily', 'low_pct_open_abs_max_daily',
       'abs_max_daily'],
      dtype='object')

In [41]:
fig = px.scatter(x=[0, 1, 2, 3, 4], y=[0, 1, 4, 9, 16])
fig.show()


ValueError: Invalid property specified for object of type plotly.graph_objs.Scatter: 'fillpattern'

Did you mean "fillcolor"?

    Valid properties:
        cliponaxis
            Determines whether or not markers and text nodes are
            clipped about the subplot axes. To show markers and
            text nodes above axis lines and tick labels, make sure
            to set `xaxis.layer` and `yaxis.layer` to *below
            traces*.
        connectgaps
            Determines whether or not gaps (i.e. {nan} or missing
            values) in the provided data arrays are connected.
        customdata
            Assigns extra data each datum. This may be useful when
            listening to hover, click and selection events. Note
            that, "scatter" traces also appends customdata items in
            the markers DOM elements
        customdatasrc
            Sets the source reference on Chart Studio Cloud for
            `customdata`.
        dx
            Sets the x coo

fig = px.bar(x=["a", "b", "c"], y=[1, 3, 2])
fig.show()

In [38]:
fig = px.box(df1, x='weekday', y='abs_max_daily'), color='weekday')

ValueError: Invalid property specified for object of type plotly.graph_objs.Scatter: 'fillpattern'

Did you mean "fillcolor"?

    Valid properties:
        cliponaxis
            Determines whether or not markers and text nodes are
            clipped about the subplot axes. To show markers and
            text nodes above axis lines and tick labels, make sure
            to set `xaxis.layer` and `yaxis.layer` to *below
            traces*.
        connectgaps
            Determines whether or not gaps (i.e. {nan} or missing
            values) in the provided data arrays are connected.
        customdata
            Assigns extra data each datum. This may be useful when
            listening to hover, click and selection events. Note
            that, "scatter" traces also appends customdata items in
            the markers DOM elements
        customdatasrc
            Sets the source reference on Chart Studio Cloud for
            `customdata`.
        dx
            Sets the x coo